In [ ]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action='ignore')


import torch
print(torch.__version__)
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

from tools import eval_summary, save_feature_importance, merge_preds

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

#### Load Data

In [ ]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

In [ ]:
train_size = len(df_train)
print(train_size)

# cat_cols = []
# fea_cols = tfidf_cols + cat_cols + cnt_cols
# fea_cols = cnt_cols + cat_cols
# fea_cols = tfidf_cols + cat_cols
# len(fea_cols)

In [ ]:
# merged_ts = '20191231T113708_5499'
# merged_ts = '20191231T162533_2022'
# merged_ts = '20191231T165424_6099'
# merged_ts = '20191231T111228_8180'
# merged_ts = '20200101T184353_5853'
# merged_ts = '20200102T005929_6546_128_49980' # 984
# merged_ts = '20200102T015155_8438_128_49980' # 986
# merged_ts = '20200102T160226_8438_275_997'
# merged_ts = '20200104T033010_10938_512_49980'
# merged_ts = '20200104T141453_13225_64_49980' #1445
# merged_ts = '20200104T143414_12439_64_49980'
# merged_ts = '20200104T151347_9528_64_49980'

# merged_ts = '20200104T033010_10938_512_49980' # 9878

merged_ts = '20200105T104739_12438_128_49980' 

merged_ts = '20200106T060355_12568_128_49980' 




train_path = 'data/df_merged_{}_train.pkl'.format(merged_ts)
test_path = 'data/df_merged_{}_test.pkl'.format(merged_ts)
dict_path = 'data/df_merged_{}_fea_dict.pkl'.format(merged_ts)

df_model = joblib.load(train_path)  
df_test = joblib.load(test_path) 
fea_dict = joblib.load(dict_path)

print('model_set\n', df_model['smishing'].value_counts())

In [ ]:
for c in [c for c in df_model.columns if 'idx' in c]:
    df_model.drop(c, axis=1, inplace=True)
    df_test.drop(c, axis=1, inplace=True)

gc.collect()

In [ ]:
zero_dict = []
# zero_dict += list(joblib.load('20200105T060507_zero_dict.pkl'))
# zero_dict += list(joblib.load('model/20200105T104918_zero_dict.pkl'))
# zero_dict += list(joblib.load('model/20200105T112414_zero_dict.pkl'))
# zero_dict += list(joblib.load('model/20200105T112414_zero_dict.pkl'))
# zero_dict += list(joblib.load('model/20200105T114935_zero_dict.pkl'))

no_fea_cols = list(set(zero_dict))
# no_fea_cols += [
#     'tfidf_word_22_0169',#_xxx 은행',
#     'fea__noun_cnt',
# ]

len(no_fea_cols)

In [ ]:
fea_cols = [c for c in df_model.columns if c not in ['smishing', 'id'] + no_fea_cols]
cat_cols = []

len(fea_cols), len(cat_cols)

In [ ]:
gc.collect()

In [ ]:
print(df_model.info())
df_model = df_model.reset_index()
df_model

#### Params

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')

initscore_filename = ''
params = {
    'boosting':'gbdt',
#     'boosting':'dart',
    'num_leaves': 7, 
#     'max_depth': 5,
    'objective': 'binary',
    'metric':'auc',
    'num_threads': 16,
    'learning_rate': 0.005,
#     'is_unbalance': True,
    'scale_pos_weight':200,
    'bagging_fraction':0.5,
    'bagging_freq':10,
    'feature_fraction':0.5,
    'initscore_filename':initscore_filename,
#     'lambda_l1':200,
#     'lambda_l2':2000,
    'device_type':'gpu',
}

data_params = {
    'max_bin':128,
#     'enable_bundle': False,
}
num_round = 20000


#### Train

In [ ]:
print('merged_ts', merged_ts, 'model_ts', model_ts)
print(params)
print(data_params)
pred_csv = []
fea_size = -1
# try_list = [3, 4, 5, 6, 7, 10, 13, 16]
try_list = [5]

for t in tqdm_notebook(try_list, total=len(try_list), desc = 'Try'):
#     fea_cols = [c for c in df_model.columns if c not in ['smishing', 'id', 'score']]
    
    if fea_size > 0:
        fea_cols = list(set(np.random.choice(fea_cols, fea_size)))
        print(t, len(fea_cols), fea_cols[:100])
    else:
        print(t, len(fea_cols))

    [df_model.drop(c, axis=1, inplace=True) for c in df_model.columns if 'smishing_' in c]
    [df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]

    n_splits = t#5
    skf = StratifiedKFold(n_splits=n_splits, random_state=8405)
    cnt = 0
    
    # for train_index, valid_index in skf.split(X, y):
    for train_index, valid_index in tqdm_notebook(skf.split(df_model['id'].values, df_model['smishing'].values), total=n_splits, desc = 'CV'):
#         if cnt < 28:
#             cnt += 1
#             continue
        print('split: \n', t, '*' * 20, cnt, '*' * 20)
        
        train_set = lgb.Dataset(df_model.loc[train_index, fea_cols].values, 
                                 label=df_model.loc[train_index, 'smishing'].values, 
                                 feature_name=fea_cols, categorical_feature=cat_cols, 
                                 free_raw_data=True, params=data_params)
        
        valid_set = lgb.Dataset(df_model.loc[valid_index, fea_cols].values, 
                                 label=df_model.loc[valid_index, 'smishing'].values, 
                                 feature_name=fea_cols, categorical_feature=cat_cols, 
                                 free_raw_data=True, params=data_params)

        print('train_set', np.unique(train_set.get_label(), return_counts=True))
        print('valid_set', np.unique(valid_set.get_label(), return_counts=True))

        bst = lgb.train(params, train_set, num_round, categorical_feature=cat_cols,
                        early_stopping_rounds=500, 
                        valid_sets=[train_set, valid_set],
                        verbose_eval=200,
                        # init_model=init_model,
                       )
        
        pred_tag = 'smishing_{}_{}__{}'.format(t, cnt, bst.best_iteration)
        print('pred_tag', pred_tag, 'Best Iteration:', bst.best_iteration)

        renamed_cols = [fea_dict[c] if c in fea_dict.keys() else c for c in fea_cols]
        
        save_feature_importance(renamed_cols, bst, 'split', f"{model_ts}_{pred_tag}")
        save_feature_importance(renamed_cols, bst, 'gain', f"{model_ts}_{pred_tag}")

        df_pred_valid = pd.DataFrame(index=valid_index, 
                               data=bst.predict(df_model.loc[valid_index, fea_cols].values),
                               columns=['score'])
        
        df_pred_valid['type'] = 'valid'
        df_pred_valid['smishing'] = df_model.loc[valid_index, 'smishing'].values
        
        df_pred_train = pd.DataFrame(index=train_index, 
                               data=bst.predict(df_model.loc[train_index, fea_cols].values),
                               columns=['score'])
        
        df_pred_train['type'] = 'train'
        df_pred_train['smishing'] = df_model.loc[train_index, 'smishing'].values
        
        
        print('valid\n', eval_summary(df_pred_valid['smishing'].values, 
                                      df_pred_valid['score'].values, cut_off=0.5))

        print('train\n', eval_summary(df_pred_train['smishing'].values, 
                                      df_pred_train['score'].values, cut_off=0.5))
        
        df_pred_model = pd.concat([df_pred_train, df_pred_valid])
              
        df_pred_model[['score', 'smishing']].to_csv('submit/{}_{}_lgb_score.csv'.format(model_ts, pred_tag), index=True)
        print('model\n', eval_summary(df_pred_model['smishing'].values, df_pred_model['score'].values, cut_off=0.5))
        
        df_test['smishing'] = bst.predict(df_test[fea_cols].values)    
        df_test[['smishing']].to_csv('submit/{}_{}_lgb.csv'.format(model_ts, pred_tag), index=True)
        pred_csv.append('submit/{}_{}_lgb.csv'.format(model_ts, pred_tag))
        
        del train_set, valid_set
        gc.collect()
        cnt = cnt + 1


#### Submit

In [ ]:
df_submit = merge_preds(pred_csv)

In [ ]:
# df_submit = df_test[['smishing']]
# df_submit['smishing'] = 0
# for csv in pred_csv:
#     print(csv)
#     df = pd.read_csv(csv, index_col=0)
#     c = csv.split('__')[0][-1]
#     df_submit['smishing_{}'.format(c)] = df['smishing']

In [ ]:
# pred_cols = [c for c in df_submit.columns if 'smishing_' in c]
# df_submit['std'] = df_submit[pred_cols].std(axis=1)
# df_submit['median'] = df_submit[pred_cols].median(axis=1)
# df_submit['mean'] = df_submit[pred_cols].mean(axis=1)

# pred_cols_n4 = [c for c in pred_cols if '_4' in c]
# df_submit['std_n4'] = df_submit[pred_cols_n4].std(axis=1)
# df_submit['median_n4'] = df_submit[pred_cols_n4].median(axis=1)
# df_submit['mean_n4'] = df_submit[pred_cols_n4].mean(axis=1)


#### Save

In [ ]:
print(model_ts)
df_submit['smishing'] = df_submit['median']
df_submit[['smishing']].to_csv('submit/{}_lgb_median.csv'.format(model_ts), index=True)

df_submit['smishing'] = df_submit['mean_n4']
df_submit[['smishing']].to_csv('submit/{}_lgb_n4.csv'.format(model_ts), index=True)

df_submit['smishing'] = df_submit['mean']
df_submit[['smishing']].to_csv('submit/{}_lgb.csv'.format(model_ts), index=True)

#### Eval

##### Feature Imp

In [ ]:
fea_imp_dict = {}
for pkl in glob.glob(f'model/{model_ts}*split.pkl'):
    for k, v in joblib.load(pkl).items():
        if k in fea_imp_dict.keys():
            fea_imp_dict[k] += v
        else:
            fea_imp_dict[k] = v
sorted(fea_imp_dict.items(), key=(lambda x:x[1]), reverse=True)

In [ ]:
non_zero_cols = [k for k, v in fea_imp_dict.items() if v != 0]
zero_cols = set(fea_imp_dict.keys()) - set(non_zero_cols)

print('non_zero_cols', len(non_zero_cols) , 'zero_cols', len(zero_cols))

zero_dict = {}
non_zero_dict = {}
for k, v in fea_dict.items():
    if v in zero_cols:
        zero_dict[k] = v
    if v in non_zero_cols:
        non_zero_dict[k] = v
    
print(joblib.dump(zero_dict, f'model/{model_ts}_zero_dict.pkl'))
print(joblib.dump(non_zero_dict, f'model/{model_ts}_non_zero_dict.pkl'))

In [ ]:
df_std = df_submit[df_submit['std'] < 1]
print(df_std.shape)
df_std.sort_values('std', ascending=False).head(30)

In [ ]:
df_submit['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
(df_submit['smishing'] * 10).astype(int).value_counts(sort=False)

In [ ]:
print(model_ts)
df_submit[['smishing']].to_csv('submit/{}_lgb.csv'.format(model_ts), index=True)